Credits to Jonathan Dekthiar (@Born2Data).
None of this is mine, except for some comments. 

In [ ]:
import os # appel système
import numpy as np # manipulations matricielles, un peu d'algèbre linéaire
import pandas as pd # manipulation de tableau, jointure SQL, etc.

Le pipeline est un tuyau dans lequel on envoie une donnée brute à l'entrée, et on récupère le résultat final à la sortie.

A l'entrée, on va donner train_users_2.csv, et on devrait obtenir ce que l'algorithme en fait. Toutes les opérations vont s'enchaîner à l'intérieur même du pipeline. On va créer une ou des fonctions, à appliquer à l'intérieur du tube, qui vont donner le modèle de sortie.

On doit déclarer ces différentes étapes : on établit une routine de nettoyage. 

In [ ]:
df_train = pd.read_csv("../input/train_users_2.csv")
df_train.sample(n=5) # Ne montre que quelques lignes choisie au hasard

On charge les données de test. 

In [ ]:
df_test = pd.read_csv("../input/test_users.csv")
df_test.sample(n=5)

On souhaite appliquer un traitement sur les colonnes du train ET du test sans avoir à écrire la ligne de code 2 fois. 
On va alors vouloir empiler les 2 tableaux.

Il faut se méfier de la colonne de première réservation

In [ ]:
#On combine les 2 tableaux

df_all = pd.concat((df_train, df_test), axis = 0, ignore_index = True)
# on importe pas l'index car pandas numérote les lignes et on ne veut pas que ça collisione
df_all.head(n=5)

In [ ]:
# On supprime la colonne de 1ère réservation qui est embêtante
df_all.drop('date_first_booking', axis = 1, inplace = True)

In [ ]:
df_all.sample(n=5)

Le standard de date n'est pas le même dans le timestamp et le date_account_created...

In [ ]:
df_all['date_account_created'] = pd.to_datetime(df_all['date_account_created'], format = '%Y-%m-%d')
df_all['timestamp_first_active'] = pd.to_datetime(df_all['timestamp_first_active'], format = '%Y%m%d%H%M%S')

df_all.sample(n=5)

On va maintenant nettoyer les valeurs aberrantes grâce à cette petite fonction.

On va considérer les personnes de - 15 ans et de + 90 ans comme des valeurs aberrantes et on va les supprimer de notre dataset.

In [ ]:
def remove_age_outliers(x, min_value=15, max_value=90): #operations logiques sur des tableaux
    if np.logical_or(x<=min_value, x>=max_value):
        return np.nan
    else:
        return x
    

In [ ]:
df_all['age'] = df_all['age'].apply(lambda x: remove_age_outliers(x) if(not np.isnan(x)) else x)
# Pandas accepte qu'on applique une fonction sur toutes les valeurs d'une ligne ou d'une colonne
# est-ce que naN est superieur ou égal à 90? Comparaison pas toujours possible

df_all['age'].head(n=20)

In [ ]:
# on remplace les NaN par -1
df_all['age'].fillna(-1, inplace=True)
df_all.head(n=10)

In [ ]:
# L'age est écrit comme n réel ! Conversion en entier.

df_all.age = df_all.age.astype(int)
df_all.age.sample(n=10)

On va maintenant chercher les lignes avec des NaN pour s'en débarasser.

In [ ]:
def check_NaN_values_in_df(df):
    for col in df: # col va être chacune des colonnes
        nan_count = df[col].isnull().sum() #nombre de valeurs nulles
        
        if nan_count != 0:
            print(col + " => " + str(nan_count) + " NaN values") #nan_count is int => string

In [ ]:
check_NaN_values_in_df(df_all)

In [ ]:
df_all['first_affiliate_tracked'].fillna(-1, inplace = True)

In [ ]:
check_NaN_values_in_df(df_all)
df_all.sample(n=5)

On enlève le timestamp qui va pas nous servir énormément et qui a un format qui ne plaît pas.

In [ ]:
df_all.drop('timestamp_first_active', axis = 1, inplace = True)
df_all.sample(n=5)

In [ ]:
# Il faut faire attention avec ce que l'on supprime. Néanmoins,
# il se peut qu'on retire de grosses informations (patterns)
df_all.drop('language', axis = 1, inplace = True)
df_all.sample(n=5)

Les utilisateurs sont les plus anciens sont particuliers : ils sont aventureux et veulent tester l'appli. Ils n'ont pas le comportement habituel et on décide de les retirer.

In [ ]:
df_all = df_all[df_all['date_account_created'] > '2013-02-01']
df_all.sample(n=5)

In [ ]:
#creation du directory si nécessaire
if not os.path.exists("output"):
    os.makedirs("output")
    
#exportation en CSV
df_all.to_csv("output/cleaned.csv", sep = ",", index = False)